In [ ]:
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

print("[0;33mConnecting...")
drive.mount('/content/gdrive')

mainpth="MyDrive"

# clear_output()

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time
import base64

if not os.path.exists("/content/gdrive"):
  print('[1;31mNot connected, using colab ...')
  time.sleep(4)
  mainpth="MyDrive"
  !mkdir -p /content/gdrive/$mainpth
  Shared_Drive=""

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/$mainpth/fm
  %cd /content/gdrive/$mainpth/fm
  !git clone -q --branch main https://github.com/kaz9112/fm-analyst.git
  !mkdir -p /content/gdrive/$mainpth/fm/fm-analyst/cache/
  os.environ['TRANSFORMERS_CACHE']=f"/content/gdrive/{mainpth}/fm/fm-analyst/cache"

with capture.capture_output() as cap:
  %cd /content/gdrive/$mainpth/fm/fm-analyst/
  !git reset --hard
  !git checkout main
  time.sleep(1)
  !rm fm23da.sh
  !git pull
  # !git checkout 7cf9c253666db592e39d3b364b2f56f3ca975a3e
  !pip install gradio==3.28.1 --no-deps -qq
  
# clear_output()

In [ ]:
print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://huggingface.co/kaz9112/fm-dep-analyst/raw/main/Dependencies/fm23da.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/'+mainpth+'/fm/fm-dep-analyst'):
    !tar -C /content/gdrive/$mainpth --zstd -xf fm_rep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  !pip install gradio==3.28.1 --no-deps -qq

# clear_output()

In [ ]:
from IPython.utils import capture
import time
import sys
import fileinput

Use_Cloudflare_Tunnel = True 

User = "sor"
Password= "sor"

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""


with capture.capture_output() as cap:
  %cd /content/gdrive/$mainpth/fm/fm-analyst/modules/
  !wget -q -O paths.py https://huggingface.co/kaz9112/fm-dep-analyst/raw/main/fm23da_files/paths.py
  !wget -q -O extras.py https://raw.githubusercontent.com/kaz9112/fm-analyst/main/modules/extras.py
  !wget -q -O sd_models.py https://raw.githubusercontent.com/kaz9112/fm-analyst/main/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://huggingface.co/kaz9112/fm-dep-analyst/raw/main/fm23da_files/blocks.py
  %cd /content/gdrive/$mainpth/fm/fm-analyst/
  
  # problem here, there are some new additions here
  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/gdrive/$mainpth/fm/fm-analyst/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/gdrive/$mainpth/fm/fm-analyst/fm23da.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/$mainpth/fm/fm-analyst/modules/extras.py
  !sed -i 's@/content/gdrive/MyDrive/fm/fm-dep-analyst@/content/gdrive/{mainpth}/fm/fm-dep-analyst@' /content/gdrive/$mainpth/fm/fm-analyst/modules/paths.py
  !sed -i 's@print(\"No module.*@@' /content/gdrive/$mainpth/fm/fm-dep-analyst/ldm/modules/diffusionmodules/model.py
  !sed -i 's@\"quicksettings\": OptionInfo(.*@"quicksettings": OptionInfo("sd_model_checkpoint,  sd_vae, CLIP_stop_at_last_layers, inpainting_mask_weight, initial_noise_multiplier", "Quicksettings list"),@' /content/gdrive/$mainpth/fm/fm-analyst/modules/shared.py
  # problem above
  
share=''
if Use_Cloudflare_Tunnel:
  with capture.capture_output() as cap:
    !pkill cloudflared
    time.sleep(4)
    !nohup cloudflared tunnel --url http://localhost:7860 > /content/srv.txt 2>&1 &
    time.sleep(4)
    !grep -o 'https[^[:space:]]*\.trycloudflare.com' /content/srv.txt >/content/srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.10/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)
            
    !rm /content/srv.txt /content/srvr.txt

else:
  share='--share'

try:
  model
  if os.path.isfile(model):
    !python /content/gdrive/$mainpth/fm/fm-analyst/fm23da.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt "$model" --xformers $auth --disable-console-progressbars
  else:
    !python /content/gdrive/$mainpth/fm/fm-analyst/fm23da.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt-dir "$model" --xformers $auth --disable-console-progressbars
except:
   !python /content/gdrive/$mainpth/fm/fm-analyst/fm23da.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers $auth --disable-console-progressbars